# Tests with AKV and PGML

In [2]:
from pgml import *
from AKV import *
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def draw_graph(model: Model):
    DG = nx.DiGraph()
    world_size = model.worlds_size
    DG.add_nodes_from(list(range(model.worlds_size)))
    DG.add_weighted_edges_from(
        [
            (i, j, model.relation[i][j])
            for i in range(world_size)
            for j in range(world_size)
            if model.relation[i][j] != 0
        ]
    )
    # labels = nx.get_edge_attributes(G, "weights")
    nx.draw(DG)

# Example 1: Faintly connected graph with mildly polarization

In [4]:
# Faintly connected graph...
NUM_AGENTS = 10
WEAK_INFLUENCE = 0.1
STRONG_INFLUENCE = 0.5
influence_graph = InfluenceGraphs.faintly(10, WEAK_INFLUENCE, STRONG_INFLUENCE)

# Not necessary for faintly connected graph, but it is necessary for any non symetric graph.
relation = [
    [influence_graph[j][i] for j in range(NUM_AGENTS)] for i in range(NUM_AGENTS)
]

# Remove self-loops.
for i in range(10):
    relation[i][i] = 0

# ...with mildly initial state.
belief_array = [InitialConfigurations.mildly(NUM_AGENTS)]
belief_array.append([1 - belief_array[0][i] for i in range(NUM_AGENTS)])

formula = Box(Variable("p"))
akv_model = AKV(belief_array, influence_graph)

# Run AKV 10 times
NUM_STEPS = 10
for i in range(NUM_STEPS):
    akv_model.overall_class_update()

# Get polarization array
# polarization = akv_model.get_polarization(k=11)

# Create a PGML model for each step
pgml_models = []
for i in range(NUM_STEPS + 1):
    pgml_models += [
        Model(
            NUM_AGENTS,
            relation,
            {"p": akv_model.states[i][0].tolist()},
            {"p": akv_model.states[i][1].tolist()},
        )
    ]

# Build table for agent
ALPHA = Box(Variable("p"))
BETA = Diamond(Variable("p"))
AGENT = 0
table = [
    {
        # "polarization": polarization[i],
        "agent valuation1": akv_model.states[i][0][AGENT],
        "agent valuation2": akv_model.states[i][1][AGENT],
        "pgml alpha valuation": ALPHA.valuation1(pgml_models[i], AGENT),
        "compare": ">"
        if ALPHA.valuation1(pgml_models[i], AGENT)
        > BETA.valuation1(pgml_models[i], AGENT)
        else "<",
        "pgml beta valuation": BETA.valuation1(pgml_models[i], AGENT),
    }
    for i in range(NUM_STEPS + 1)
]

# Dataframe
pd.DataFrame(table)

,agent valuation1,agent valuation2,pgml alpha valuation,compare,pgml beta valuation
0,0.200000,0.800000,0.240000,<,0.360000
1,0.350000,0.650000,0.370000,<,0.430000
2,0.425000,0.575000,0.435000,<,0.465000
3,0.462500,0.537500,0.467500,<,0.482500
4,0.481250,0.518750,0.483750,<,0.491250
5,0.490625,0.509375,0.491875,<,0.495625
6,0.495312,0.504687,0.495938,<,0.497812
7,0.497656,0.502344,0.497969,<,0.498906
8,0.498828,0.501172,0.498984,<,0.499453
9,0.499414,0.500586,0.499492,<,0.499727


## Example 2

In [8]:
# Malleable influencers graph...
INFLUENCERS_OUTGOING_VALUE_FIRST = 0.8
INFLUENCERS_OUTGOING_VALUE_SECOND = 0.5
INFLUENCERS_INCOMING_VALUE_FIRST = 0.1
INFLUENCERS_INCOMING_VALUE_SECOND = 0.1
OTHERS_INFLUENCE = 0.2
influence_graph = InfluenceGraphs.two_influencers_unbalanced(
    NUM_AGENTS,
    INFLUENCERS_OUTGOING_VALUE_FIRST,
    INFLUENCERS_OUTGOING_VALUE_SECOND,
    INFLUENCERS_INCOMING_VALUE_FIRST,
    INFLUENCERS_INCOMING_VALUE_SECOND,
    OTHERS_INFLUENCE,
)

# Not necessary for faintly connected graph, but it is necessary for any non symetric graph.
relation = [
    [influence_graph[j][i] for j in range(NUM_AGENTS)] for i in range(NUM_AGENTS)
]

# Remove self-loops.
for i in range(10):
    relation[i][i] = 0

# ...with mildly polarized initial state.
belief_array = [InitialConfigurations.mildly(NUM_AGENTS)]
belief_array.append([1 - belief_array[0][i] for i in range(NUM_AGENTS)])

formula = Box(Variable("p"))
akv_model = AKV(belief_array, influence_graph)

# Run AKV 10 times
NUM_STEPS = 10
for i in range(NUM_STEPS):
    akv_model.overall_class_update()

# Get polarization array
# polarization = akv_model.get_polarization(k=11)

# Create a PGML model for each step
pgml_models = []
for i in range(NUM_STEPS + 1):
    pgml_models += [
        Model(
            NUM_AGENTS,
            relation,
            {"p": akv_model.states[i][0].tolist()},
            {"p": akv_model.states[i][1].tolist()},
        )
    ]

In [10]:
# Build table for agent
ALPHA = Box(Variable("p"))
BETA = Diamond(Variable("p"))
AGENT = 1
table = [
    {
        # "polarization": polarization[i],
        "agent valuation1": akv_model.states[i][0][AGENT],
        "agent valuation2": akv_model.states[i][1][AGENT],
        "pgml alpha valuation": ALPHA.valuation1(pgml_models[i], AGENT),
        "compare": ">"
        if ALPHA.valuation1(pgml_models[i], AGENT)
        > BETA.valuation1(pgml_models[i], AGENT)
        else "<",
        "pgml beta valuation": BETA.valuation1(pgml_models[i], AGENT),
    }
    for i in range(NUM_STEPS + 1)
]

# Dataframe
pd.DataFrame(table)

,agent valuation1,agent valuation2,pgml alpha valuation,compare,pgml beta valuation
0,0.240000,0.760000,0.200000,<,0.5
1,0.266000,0.552000,0.230000,<,0.5
2,0.280300,0.437600,0.246500,<,0.5
3,0.288165,0.374680,0.255575,<,0.5
4,0.292491,0.340074,0.260566,<,0.5
5,0.294870,0.321041,0.263311,<,0.5
6,0.296178,0.310572,0.264821,<,0.5
7,0.296898,0.304815,0.265652,<,0.5
8,0.297294,0.301648,0.266108,<,0.5
9,0.297512,0.299906,0.266360,<,0.5


In [6]:
akv_model.states

array([[[0.2       , 0.24      , 0.28      , 0.32      , 0.36      ,
         0.6       , 0.64      , 0.68      , 0.72      , 0.76      ],
        [0.8       , 0.76      , 0.72      , 0.68      , 0.64      ,
         0.4       , 0.36      , 0.32      , 0.28      , 0.24      ]],

       [[0.23      , 0.266     , 0.302     , 0.338     , 0.374     ,
         0.59      , 0.626     , 0.662     , 0.698     , 0.734     ],
        [0.56      , 0.552     , 0.544     , 0.536     , 0.528     ,
         0.48      , 0.472     , 0.464     , 0.456     , 0.448     ]],

       [[0.2465    , 0.2803    , 0.3141    , 0.3479    , 0.3817    ,
         0.5845    , 0.6183    , 0.6521    , 0.6859    , 0.7197    ],
        [0.428     , 0.4376    , 0.4472    , 0.4568    , 0.4664    ,
         0.524     , 0.5336    , 0.5432    , 0.5528    , 0.5624    ]],

       [[0.255575  , 0.288165  , 0.320755  , 0.353345  , 0.385935  ,
         0.581475  , 0.614065  , 0.646655  , 0.679245  , 0.711835  ],
        [0.3554    , 

In [7]:
akv_model.influence_graph

array([[1. , 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.1],
       [0.1, 1. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.1, 0.2, 1. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.1, 0.2, 0.2, 1. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.1, 0.2, 0.2, 0.2, 1. , 0.2, 0.2, 0.2, 0.2, 0.1],
       [0.1, 0.2, 0.2, 0.2, 0.2, 1. , 0.2, 0.2, 0.2, 0.1],
       [0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 1. , 0.2, 0.2, 0.1],
       [0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1. , 0.2, 0.1],
       [0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 1. , 0.1],
       [0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. ]])